In [36]:
import requests
import pandas as pd
from collections import defaultdict
from itertools import combinations
import networkx as nx
import time
import netwulf as nw
import matplotlib.pyplot as plt
import numpy as np

In [37]:
def read_csv_file(file_path):
    try:
        df = pd.read_csv(file_path,sep=";")
        return df
    except Exception as e:
        print(f"Failed to read CSV: {e}")
        return None

In [38]:
characters = read_csv_file('../data/webscrape_script_char.csv')
characters

,name,species,affiliation,gender,pronouns,hair,skin,homeworld,death,height,eyes,class,mass,apprentices,birth,families,parents,children,matched_name
0,2r-series medical droid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medical droid,NaN,NaN,NaN,NaN,NaN,NaN,medical droid
1,aat driver battle droid,NaN,Confederacy of Independent Systems,Masculine programming,NaN,NaN,NaN,NaN,NaN,"1.93 meters (6 ft, 4 in)",NaN,Battle droid,NaN,NaN,NaN,NaN,NaN,NaN,battle droid
2,b2-ha super battle droid,NaN,Confederacy of Independent Systems,Masculine programming,NaN,NaN,NaN,NaN,NaN,"1.93 meters (6 ft, 4 in)",NaN,Battle droid,NaN,NaN,NaN,NaN,NaN,NaN,super battle droid 2
3,c-3po,NaN,Skywalker family,Masculine programming,He/him,NaN,NaN,Tatooine,"0 ABY,","1.77 meters (5 ft, 10 in)",NaN,Protocol droid,75 kilograms,NaN,Between 34 BBY and 32 BBY,NaN,NaN,NaN,c-3po
4,c5 bartender droid,NaN,Alisandre Hotel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bartender droid,NaN,NaN,NaN,NaN,NaN,NaN,bartender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,salbee,NaN,Sarkin Enneb's criminal group,Female,She/her,NaN,NaN,NaN,NaN,Taller than 1.58 meters,NaN,NaN,NaN,NaN,c. 31 BBY,NaN,NaN,NaN,sabe
161,unidentified sith emperor,NaN,Sith Empire,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,emperor
162,bobbie,NaN,NaN,Feminine programming,NaN,NaN,NaN,NaN,NaN,1.7 meters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hobbie
163,gungan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 meters,NaN,Amphibian,75 kg (165lbs),NaN,NaN,NaN,NaN,NaN,gungan lookout


In [39]:
def call_api(url, params=None, headers=None, method="GET", data=None):
    try:
        response = requests.request(method, url, params=params, headers=headers, json=data)
        response.raise_for_status()  # Raises HTTPError for bad responses
        return response.json()  # or response.text if it's not JSON
    except requests.RequestException as e:
        print(f"API call failed: {e}")
        return None

In [40]:
# Make the API call
url = "https://swapi.info/api/people"
response = call_api(url)

# Define mapping from API to CSV
api_to_csv_map = {
    "name": "name",
    "species": "species",          
    "gender": "gender",
    "hair_color": "hair",
    "skin_color": "skin",
    "homeworld": "homeworld",        
    "eye_color": "eyes"
}

n = 0
c = 0
# Loop through characters from API and update CSV (we know we have a API response)
for char in response:
    name = char.get("name")
    
    # Clean the name to match df name format
    name = name.lower().strip().replace('"', '')
    if name in characters["name"].values:
        print(f"{name} found in CSV.")
        n += 1
        idx = characters[characters["name"] == name].index[0]
        for api_field, csv_field in api_to_csv_map.items():
            current_value = characters.at[idx, csv_field]

            # Only update if df field is missing or empty
            if pd.isna(current_value) or current_value == "":
                value = char.get(api_field)
                print(f"Updating {csv_field} for {name}: {value}")
                c+=1

                if isinstance(value, list):
                    value = ", ".join(value)
                if value:
                    characters.at[idx, csv_field] = value

print(f"------------- \n {n} characters updated in CSV.")
print(f"------------- \n {c} values updated in CSV.")

luke skywalker found in CSV.
c-3po found in CSV.
Updating species for c-3po: ['https://swapi.info/api/species/2']
Updating hair for c-3po: n/a
Updating skin for c-3po: gold
Updating eyes for c-3po: yellow
owen lars found in CSV.
beru whitesun lars found in CSV.
biggs darklighter found in CSV.
obi-wan kenobi found in CSV.
anakin skywalker found in CSV.
han solo found in CSV.
greedo found in CSV.
Updating hair for greedo: n/a
jabba desilijic tiure found in CSV.
Updating hair for jabba desilijic tiure: n/a
wedge antilles found in CSV.
jek tono porkins found in CSV.
yoda found in CSV.
boba fett found in CSV.
mon mothma found in CSV.
nute gunray found in CSV.
Updating hair for nute gunray: none
padmé amidala found in CSV.
jar jar binks found in CSV.
Updating hair for jar jar binks: none
ric olié found in CSV.
Updating skin for ric olié: fair
watto found in CSV.
sebulba found in CSV.
Updating hair for sebulba: none
bib fortuna found in CSV.
Updating hair for bib fortuna: none
mace windu foun

In [41]:
characters.to_csv('../data/Refined_characters.csv', index=False)

---